# Spark SQL

## Catalog API

In [1]:
for table in spark.catalog.listTables(): print(table)

Table(name=u'accounts', database=u'default', description=None, tableType=u'EXTERNAL', isTemporary=False)
Table(name=u'part_test', database=u'default', description=None, tableType=u'MANAGED', isTemporary=False)


In [2]:
for column in spark.catalog.listColumns("accounts"): print(column)

Column(name=u'acct_num', description=None, dataType=u'int', nullable=True, isPartition=False, isBucket=False)
Column(name=u'acct_create_dt', description=None, dataType=u'timestamp', nullable=True, isPartition=False, isBucket=False)
Column(name=u'acct_close_dt', description=None, dataType=u'timestamp', nullable=True, isPartition=False, isBucket=False)
Column(name=u'first_name', description=None, dataType=u'string', nullable=True, isPartition=False, isBucket=False)
Column(name=u'last_name', description=None, dataType=u'string', nullable=True, isPartition=False, isBucket=False)
Column(name=u'address', description=None, dataType=u'string', nullable=True, isPartition=False, isBucket=False)
Column(name=u'city', description=None, dataType=u'string', nullable=True, isPartition=False, isBucket=False)
Column(name=u'state', description=None, dataType=u'string', nullable=True, isPartition=False, isBucket=False)
Column(name=u'zipcode', description=None, dataType=u'string', nullable=True, isPartitio

In [3]:
df = spark.read.table("accounts")
df.printSchema()

root
 |-- acct_num: integer (nullable = true)
 |-- acct_create_dt: timestamp (nullable = true)
 |-- acct_close_dt: timestamp (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- address: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- zipcode: string (nullable = true)
 |-- phone_number: string (nullable = true)
 |-- created: timestamp (nullable = true)
 |-- modified: timestamp (nullable = true)



## table sql query

In [4]:
fldf = spark.sql("SELECT first_name,last_name FROM accounts")

In [5]:
fldf.printSchema()

root
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)



In [6]:
fldf.show(5)

+----------+---------+
|first_name|last_name|
+----------+---------+
|    Donald|   Becton|
|     Donna|    Jones|
|    Dorthy| Chalmers|
|     Leila|  Spencer|
|     Anita| Laughlin|
+----------+---------+
only showing top 5 rows



In [7]:
fldf.explain()

== Physical Plan ==
HiveTableScan [first_name#65, last_name#66], MetastoreRelation default, accounts


In [8]:
fldf2 = df.select("first_name", "last_name")

In [9]:
fldf2.explain()

== Physical Plan ==
HiveTableScan [first_name#40, last_name#41], MetastoreRelation default, accounts


In [10]:
fldf2.printSchema()

root
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)



In [11]:
fldf2.show(5)

+----------+---------+
|first_name|last_name|
+----------+---------+
|    Donald|   Becton|
|     Donna|    Jones|
|    Dorthy| Chalmers|
|     Leila|  Spencer|
|     Anita| Laughlin|
+----------+---------+
only showing top 5 rows



## view

In [12]:
adf = spark.read.option("header","true").option("inferSchema","true").csv("/loudacre/accountdevice")

In [13]:
adf.printSchema()

root
 |-- id: integer (nullable = true)
 |-- account_id: integer (nullable = true)
 |-- device_id: integer (nullable = true)
 |-- activation_date: long (nullable = true)
 |-- account_device_id: string (nullable = true)



In [14]:
adf.createTempView("account_dev")

In [15]:
for table in spark.catalog.listTables(): print(table)

Table(name=u'accounts', database=u'default', description=None, tableType=u'EXTERNAL', isTemporary=False)
Table(name=u'part_test', database=u'default', description=None, tableType=u'MANAGED', isTemporary=False)
Table(name=u'account_dev', database=None, description=None, tableType=u'TEMPORARY', isTemporary=True)


In [16]:
spark.sql("SELECT * FROM account_dev LIMIT 5").show()

+-----+----------+---------+---------------+--------------------+
|   id|account_id|device_id|activation_date|   account_device_id|
+-----+----------+---------+---------------+--------------------+
|48692|     32443|       29|  1393242509000|7351fed1-f344-4cd...|
|48693|     32444|        4|  1353649861000|6da22278-ff7a-461...|
|48694|     32445|        9|  1331819465000|cb993b85-6775-407...|
|48695|     32446|       43|  1336860950000|48ea2c09-a0df-4d1...|
|48696|     32446|       29|  1383650663000|4b49c0a6-d141-42e...|
+-----+----------+---------+---------------+--------------------+



In [17]:
ndf = spark.sql("SELECT acct_num, first_name, last_name, account_device_id FROM accounts JOIN account_dev ON acct_num = account_id")

In [18]:
ndf.show()

+--------+----------+---------+--------------------+
|acct_num|first_name|last_name|   account_device_id|
+--------+----------+---------+--------------------+
|       1|    Donald|   Becton|20b671b4-2467-42d...|
|       1|    Donald|   Becton|7eb61253-55cd-430...|
|       2|     Donna|    Jones|658e3fac-8e85-40e...|
|       3|    Dorthy| Chalmers|fb674b1b-135e-408...|
|       4|     Leila|  Spencer|c0b0c922-7fd4-46b...|
|       5|     Anita| Laughlin|e054dafa-88b8-41e...|
|       5|     Anita| Laughlin|b72a7271-864e-46d...|
|       6|    Stevie|   Bridge|7d11b8e3-d67d-45f...|
|       7|     David|   Eggers|98428e3a-bfb0-487...|
|       8|   Dorothy|  Koopman|c49c4f82-9e22-4b3...|
|       8|   Dorothy|  Koopman|58830f32-8a63-4af...|
|       9|      Kara|     Kohl|5266e928-9304-4ee...|
|       9|      Kara|     Kohl|45fe71c6-c56f-435...|
|      10|     Diane|   Nelson|27bdf2f3-82a2-48d...|
|      10|     Diane|   Nelson|ece98119-a036-400...|
|      11|    Robert|   Fisher|678dc546-57ca-4

In [19]:
ndf.write.option("path","/loudacre/name_dev").saveAsTable("name_dev")

In [20]:
for column in spark.catalog.listColumns("name_dev"): print(column)

Column(name=u'acct_num', description=None, dataType=u'int', nullable=True, isPartition=False, isBucket=False)
Column(name=u'first_name', description=None, dataType=u'string', nullable=True, isPartition=False, isBucket=False)
Column(name=u'last_name', description=None, dataType=u'string', nullable=True, isPartition=False, isBucket=False)
Column(name=u'account_device_id', description=None, dataType=u'string', nullable=True, isPartition=False, isBucket=False)
